In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import tensorflow as tf
import random
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Carregando os dados de treinamento e teste
train_df = pd.read_csv("train_audience.csv")  # Carregando dados de treino
train_df = train_df.groupby('Rating', group_keys=False).apply(lambda x: x.sample(1500)).reset_index(drop=True)

test_df = pd.read_csv("test_audience.csv")
test_df = test_df.groupby('Rating', group_keys=False).apply(lambda x: x.sample(300)).reset_index(drop=True)

distinct_rating = train_df["Rating"].unique()
distinct_rating_dict = {}
for index, label in enumerate(distinct_rating):
    distinct_rating_dict[label] = index


distinct_rating_test = test_df["Rating"].unique()
distinct_rating_dict_test = {}
for index, label in enumerate(distinct_rating_test):
    distinct_rating_dict_test[label] = index

train_df['Rating'] = train_df["Rating"].replace(distinct_rating_dict)
test_df['Rating'] = test_df["Rating"].replace(distinct_rating_dict_test)

vectorizer = TfidfVectorizer(stop_words='english', max_features = 3000) 
x_train = vectorizer.fit_transform(train_df['Review'])
y_train = train_df['Rating']

x_test = vectorizer.fit_transform(train_df['Review'])
y_test = test_df['Rating']

2024-11-24 15:32:50.003721: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-24 15:32:50.011195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732473170.020171   89700 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732473170.022756   89700 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 15:32:50.032003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns


def evaluate_metrics(y, p):
    # Cálculo das métricas de acerto.
    print('Accuracy:', round(metrics.accuracy_score(y, p),3))
    print('Accuracy (balanced):', round(metrics.balanced_accuracy_score(y, p), 3))

    # Calculo da matriz de confusão.
    c = metrics.confusion_matrix(y, p)
    r = c / c.sum(axis=1, keepdims=True)

    plt.figure(figsize=(10, 8))
    sns.heatmap(r, annot=True, fmt=".2f", cmap="Blues")
    plt.xlabel("Classe predita")
    plt.ylabel("Classe verdadeira")
    plt.title("Matriz de confusão relativa")
    plt.show()

In [3]:
from sklearn.ensemble import RandomForestClassifier

rndForest = RandomForestClassifier(max_depth=8, n_estimators=800, max_features=800)
rndForest.fit(x_train, y_train)
p_test = rndForest.predict(x_test)

In [6]:
print(len(y_test))
print(len(p_test))

3000
15000


In [5]:
evaluate_metrics(y_test, p_test)

ValueError: Found input variables with inconsistent numbers of samples: [3000, 15000]

In [ ]:
print(metrics.classification_report(y_test, p_test))